In [1]:
# MIA Pneumonia project model

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy(policy="mixed_float16")
tf.random.set_seed(329)


In [7]:
# Loading dataset

train_dir = 'Input/chest_xray/train'
test_dir = 'Input/chest_xray/test'
val_dir = 'Input/chest_xray/val'
IMG_SIZE = (224,224)
BATCH_SIZE = 32

train_gen = ImageDataGenerator(
                rescale=1. / 255,
                horizontal_flip=True)
val_gen = ImageDataGenerator(rescale=1. / 255)
test_gen = ImageDataGenerator(rescale=1. / 255)

In [8]:
# Preprocessing

train_data= train_gen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle = True)
val_data = val_gen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary')
test_data = test_gen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [9]:
# Defining Callbacks

model_checkpoint = tf.keras.callbacks.ModelCheckpoint('cp.weights.h5',
                                                      monitor="val_loss",
                                                      save_best_only=True,
                                                      save_weights_only=True)
early_stopping = tf.keras.callbacks.EarlyStopping(
                              monitor='val_loss',
                              patience=5,
                              min_delta=1e-7,
                              restore_best_weights=True,
                              )


In [12]:
# Transfer Learning using DenseNet121 and feature extraction

input_shape=(224,224,3)

base_model = tf.keras.applications.DenseNet121(include_top=False,input_shape=input_shape)
base_model.trainable = False
inputs = tf.keras.layers.Input(shape=input_shape,name='input_layer')

base_model = base_model(inputs)
base_model = tf.keras.layers.GlobalAveragePooling2D()(base_model)

outputs = tf.keras.layers.Dense(1, activation = 'sigmoid',dtype=tf.float32)(base_model)

model = tf.keras.models.Model(inputs = inputs, outputs = outputs)

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 9s 0us/step


In [15]:
# We use Adam optimizer

model.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    metrics=['binary_accuracy']
)

In [16]:
# To get a summary of model

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ densenet121 (Functional)        │ (None, 7, 7, 1024)     │     7,037,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1024)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,038,529 (26.85 MB)

 Trainable params: 1,025 (4.00 KB)

 Non-trainable params: 7,037,504 (26.85 MB)

In [17]:
# Fitting the model and defining number of epochs

history = model.fit(train_data,
         epochs=20,
         validation_data = test_data,
         callbacks = [model_checkpoint,early_stopping])

Epoch 1/20


e:\Program Files\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


163/163 ━━━━━━━━━━━━━━━━━━━━ 455s 3s/step - binary_accuracy: 0.7394 - loss: 0.5495 - val_binary_accuracy: 0.6971 - val_loss: 0.5292
Epoch 2/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 425s 3s/step - binary_accuracy: 0.7999 - loss: 0.4167 - val_binary_accuracy: 0.7292 - val_loss: 0.4722
Epoch 3/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 424s 3s/step - binary_accuracy: 0.8620 - loss: 0.3449 - val_binary_accuracy: 0.7532 - val_loss: 0.4393
Epoch 4/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 417s 3s/step - binary_accuracy: 0.8906 - loss: 0.2931 - val_binary_accuracy: 0.7740 - val_loss: 0.4146
Epoch 5/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 408s 2s/step - binary_accuracy: 0.9183 - loss: 0.2586 - val_binary_accuracy: 0.7853 - val_loss: 0.4067
Epoch 6/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 410s 3s/step - binary_accuracy: 0.9253 - loss: 0.2386 - val_binary_accuracy: 0.8013 - val_loss: 0.3903
Epoch 7/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 409s 3s/step - binary_accuracy: 0.9319 - loss: 0.2214 - val_binary_accuracy: 0.8029 - val_loss: 0.3892
Epoch 8/20

In [11]:
loss, accuracy = model.evaluate(test_data)
print('The accuracy of the model on test dataset is',
    np.round(accuracy*100))
print('The loss of the model on test dataset is',
    np.round(loss*100))

20/20 ━━━━━━━━━━━━━━━━━━━━ 45s 2s/step - binary_accuracy: 0.8155 - loss: 0.3757
The accuracy of the model on test dataset is 83.0
The loss of the model on test dataset is 36.0


In [5]:
# Saving the weights for future use

model.save('pmod.keras')